In [1]:
from geolifeclef.utils import get_spark

spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 20:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 20:20:08 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [27]:
po_df = spark.read.csv(
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/PresenceOnlyOccurrences/GLC24-PO-metadata-train.csv",
    header=True,
    inferSchema=True,
)

In [28]:
po_df.show()

+----+-----+----+--------+---------+-----------------+---------+----------+---------+---------+--------+
|year|month| day|     lat|      lon|geoUncertaintyInM|taxonRank|      date|dayOfYear|speciesId|surveyId|
+----+-----+----+--------+---------+-----------------+---------+----------+---------+---------+--------+
|2019|  5.0| 5.0|43.74605| 1.573057|              6.0|  SPECIES|2019-05-05|      125|   3383.0|       1|
|2021|  3.0|17.0|42.12559| 0.314948|              5.0|  SPECIES|2021-03-17|       76|   1152.0|       2|
|2021|  6.0| 5.0| 48.2952|-0.934518|             24.9|  SPECIES|2021-06-05|      156|   6772.0|       3|
|2021|  6.0| 9.0|53.63367|-2.644535|              8.0|  SPECIES|2021-06-09|      160|   3318.0|       4|
|2021|  4.0| 1.0|49.79471| 7.925086|             15.0|  SPECIES|2021-04-01|       91|   3374.0|       5|
|2021|  7.0|23.0|49.70944|  6.79338|             16.0|  SPECIES|2021-07-23|      204|   6293.0|       6|
|2021|  8.0|27.0|42.23383| -2.11564|              6.5| 

In [5]:
!gsutil ls gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/*PO*

gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-blue.csv
gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-green.csv
gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-nir.csv
gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-red.csv
gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-swir1.csv
gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-swir2.csv


In [29]:
red = spark.read.csv(
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-red.csv",
    header=True,
    inferSchema=True,
)

In [44]:
import pyspark.sql.functions as F

In [51]:
def filter_region(ref_df, input_df, min_lat, max_lat, min_lon, max_lon):
    survey_Ids = ref_df.filter(
        (ref_df["lat"] > min_lat)
        & (ref_df["lat"] <= max_lat)
        & (ref_df["lon"] <= max_lon)
        & (ref_df["lon"] >= min_lon)
    ).select("surveyId")
    filtered_df = input_df.join(survey_Ids, "surveyId")
    columns = [
        F.col(column_name)
        for column_name in filtered_df.columns
        if column_name != "surveyId"
    ]
    filtered_df = filtered_df.withColumn("TimeSeries", F.array(columns))
    filtered_df = filtered_df.select("surveyId", "TimeSeries")
    return filtered_df

In [52]:
format_df = filter_region(po_df, red, 52, 54, 5, 7)

In [53]:
format_df.show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|surveyId|TimeSeries                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [49]:
import os
from pathlib import Path

In [57]:
tss = [
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-blue.csv",
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-green.csv",
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-nir.csv",
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-red.csv",
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-swir1.csv",
    "gs://dsgt-clef-geolifeclef-2024/data/downloaded/2024/SatelliteTimeSeries/GLC24-PO-train-landsat-time-series-swir2.csv",
]
for ts in tss:
    ts_name = Path(ts).stem + ".parquet"
    ts_df = spark.read.csv(ts, header=True, inferSchema=True)
    ts_df = filter_region(po_df, ts_df, 52, 54, 5, 7)
    ts_df.write.mode("overwrite").parquet(
        "gs://dsgt-clef-geolifeclef-2024/data/processed/sample_time_series/" + ts_name,
    )